In [ ]:
import numpy as np
from keras import utils, callbacks
from keras.models import Sequential
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split, StratifiedKFold
#from keras.preprocessing.image import load_img, img_to_array
from skimage.transform import resize
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_directory = "../input/asl-alphabet/asl_alphabet_train/asl_alphabet_train"
#test_directory = "../input/asl-alphabet/asl_alphabet_test/asl_alphabet_test"
classes = 29
batch = 256
epochs = 50
lnr = 0.0001
gen = ImageDataGenerator(rescale=1./255, validation_split=0.1) 
train = gen.flow_from_directory("../input/asl-alphabet/asl_alphabet_train/asl_alphabet_train", 
                                                            target_size=(64, 64), subset="training")
val = gen.flow_from_directory("../input/asl-alphabet/asl_alphabet_train/asl_alphabet_train", 
                                                            target_size=(64, 64), subset="validation")

adam = Adam(learning_rate=lnr)
model = Sequential()
model.add(VGG16(weights='imagenet', include_top=False, input_shape=(64, 64, 3)))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(29, activation='softmax'))

model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'] )

earlystopping = callbacks.EarlyStopping(monitor="val_loss", mode="min", 
                                        patience=5, restore_best_weights = True)

history = model.fit(train, validation_data = val, epochs=epochs, shuffle=True, verbose=1, 
                    callbacks = [earlystopping])

test = ImageDataGenerator(rescale=1./255).flow_from_directory("../input/asl-alphabet/asl_alphabet_test", 
                                                            target_size=(64, 64), class_mode=None)
pred = model.predict(test)
pred = np.argmax(pred, axis=1)

model.save('final_model.h5')